In [4]:
# ============================================================
# 02_crop_from_clean.ipynb
# Crop images based on bounding boxes from images_clean.csv
# 根据 images_clean.csv 中的坐标裁剪服装区域
# ============================================================

import os
from pathlib import Path
import pandas as pd

# === 📁 Step 1: 路径配置 (修改为你自己的 data 目录)
DATA_DIR  = r"D:\Icey\tcd\notebooks\dMining\Group Project\workflow\data"
CSV_CLEAN = os.path.join(DATA_DIR, "images_clean.csv")
IMG_ROOT  = os.path.join(DATA_DIR, "img")              # 原始图像文件夹
CROP_DIR  = os.path.join(DATA_DIR, "cropped_images")   # 输出目录

# === 确保目录存在
os.makedirs(CROP_DIR, exist_ok=True)

# === 打印检查信息
print("[INFO] DATA_DIR :", DATA_DIR)
print("[INFO] CSV_CLEAN:", CSV_CLEAN)
print("[INFO] IMG_ROOT :", IMG_ROOT)
print("[INFO] CROP_DIR :", CROP_DIR)

# === 检查文件是否存在
if not os.path.exists(CSV_CLEAN):
    raise FileNotFoundError(f"❌ 找不到 {CSV_CLEAN} ，请先运行 00_build_light_index.ipynb 生成。")
if not os.path.exists(IMG_ROOT):
    raise FileNotFoundError(f"❌ 找不到原始图片目录 {IMG_ROOT} ，请检查 img 文件夹路径。")

print("✅ Path check passed.")


[INFO] DATA_DIR : D:\Icey\tcd\notebooks\dMining\Group Project\workflow\data
[INFO] CSV_CLEAN: D:\Icey\tcd\notebooks\dMining\Group Project\workflow\data\images_clean.csv
[INFO] IMG_ROOT : D:\Icey\tcd\notebooks\dMining\Group Project\workflow\data\img
[INFO] CROP_DIR : D:\Icey\tcd\notebooks\dMining\Group Project\workflow\data\cropped_images
✅ Path check passed.


In [6]:
# === Cell 02 Load CSV (your version structure) ===
import os
import pandas as pd

df = pd.read_csv(CSV_CLEAN)
print(f"✅ Loaded {len(df)} rows")
print("Columns:", list(df.columns))

# --- Check essential columns ---
required_cols = ['image_path', 'eval_split', 'item_id', 'x1', 'y1', 'x2', 'y2']
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise KeyError(f"❌ Missing columns in CSV: {missing}")

# --- Convert to absolute image paths ---
def to_abs_path(rel_path):
    p = str(rel_path).replace("\\", "/")
    if os.path.isabs(p):
        return p
    return os.path.join(IMG_ROOT, p)

df["image_abs"] = df["image_path"].apply(to_abs_path)

# --- Keep only existing images ---
df = df[df["image_abs"].apply(os.path.exists)].reset_index(drop=True)
print(f"✅ Valid images: {len(df)} found.")
df.head(3)


✅ Loaded 52712 rows
Columns: ['image_path', 'eval_split', 'item_id_eval', 'item_id', 'x1', 'y1', 'x2', 'y2']
✅ Valid images: 52712 found.


,image_path,eval_split,item_id_eval,item_id,x1,y1,x2,y2,image_abs
0,WOMEN/Dresses/id_00000002/02_1_front.jpg,train,id_00000002,id_00000002,65,45,233,252,D:\Icey\tcd\notebooks\dMining\Group Project\wo...
1,WOMEN/Dresses/id_00000002/02_2_side.jpg,train,id_00000002,id_00000002,112,41,168,247,D:\Icey\tcd\notebooks\dMining\Group Project\wo...
2,WOMEN/Dresses/id_00000002/02_4_full.jpg,train,id_00000002,id_00000002,89,34,169,167,D:\Icey\tcd\notebooks\dMining\Group Project\wo...


In [11]:
# === Crop images (preserve subfolders) ===
from PIL import Image
from tqdm import tqdm
import os

print(f"🚀 Start cropping {len(df)} images...")

success, skipped = 0, 0

for _, row in tqdm(df.iterrows(), total=len(df)):
    src_path = row["image_abs"]

    # 用相对 IMG_ROOT 的路径保存：cropped_images/相对子路径
    rel_path = os.path.relpath(src_path, IMG_ROOT)  # 比如 WOMEN/Dresses/id_.../02_1_front.jpg
    save_path = os.path.join(CROP_DIR, rel_path)

    # 确保子目录存在
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    try:
        with Image.open(src_path) as img:
            w, h = img.size
            x1, y1, x2, y2 = map(int, [row["x1"], row["y1"], row["x2"], row["y2"]])
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(w, x2), min(h, y2)
            if x2 <= x1 or y2 <= y1:
                raise ValueError("Invalid bbox")

            cropped = img.crop((x1, y1, x2, y2))
            cropped.save(save_path)
            success += 1
    except Exception as e:
        skipped += 1
        print(f"⚠️ Skip {src_path}: {e}")

print(f"\n✅ Done. Saved {success} crops, skipped {skipped}.")


🚀 Start cropping 52712 images...


100%|██████████| 52712/52712 [01:20<00:00, 652.75it/s]


✅ Done. Saved 52712 crops, skipped 0.


In [12]:
import glob
cropped_files = glob.glob(os.path.join(CROP_DIR, "**", "*.jpg"), recursive=True)
print(f"✅ Cropped images count: {len(cropped_files)}")


✅ Cropped images count: 53092
